# GPS Positioning

In [1]:
import numpy as np
from itertools import combinations
import math
from numpy import linalg as LA
import pandas as pd

# I.) Introduction

# II.) Methodology

# III.) Computer Experiments/Code

# Question 1

In [2]:
# get Reciever Position by satellites position and tansmission time
def RecieverPosition(Pos_SatelsMatrix, Transmission_t, InitialVector=np.array([0, 0, 6370, 0]), N=10):   
    A = Pos_SatelsMatrix[:,0] #row 1
    B = Pos_SatelsMatrix[:,1] #row 2
    C = Pos_SatelsMatrix[:,2] #row 3
    
    CurrentVector = InitialVector
    c = 299792.458 #speed of light
    F = np.zeros(4)      
    DF = np.zeros((4,4))
    
    for i in range(N): 
        x = CurrentVector[0] #1st element
        y = CurrentVector[1] #2nd element
        z = CurrentVector[2] #3rd element
        d = CurrentVector[3] #4th element               
       # functions of (x,y,z,d) 
        for i in range(len(InitialVector)):
            F[i] = ((x - A[i])**2 + (y-B[i])**2 + (z-C[i])**2 - (c*(Transmission_t[i] - d))**2)    
            DF[i][0] = (2*(x-A[i]))
            DF[i][1] = (2*(y-B[i]))
            DF[i][2] = (2*(z-C[i]))
            DF[i][3] = (2*c**2*(Transmission_t[i]-d))
    
        # calculate with Multivariate Newton’s Method
        # Formula: X(i+1) = Xi- DF(Xi)^(-1)*F(Xi)
        CurrentVector = CurrentVector - np.dot(np.linalg.inv(DF),F)       
    return CurrentVector

In [3]:
# define constant value satellites position and transmission time
Pos_SatelsMatrix = np.array([[15600, 7540, 20140], [18760, 2750, 18610], [17610, 14630, 13480], [19170, 610, 18390]])
Transmission_t = np.array([0.07074, 0.07220, 0.07690, 0.07242])
InitialVector = np.array([0, 0, 6370, 0])

# get Reciever Position with given constant value
RecieverPos = RecieverPosition(Pos_SatelsMatrix, Transmission_t, InitialVector)
print('RecieverPosition',RecieverPos)

RecieverPosition [-4.17727096e+01 -1.67891941e+01  6.37005956e+03 -3.20156583e-03]


# Question 2

In [4]:
# satellite positions in km and travel times in seconds
def SatellitePosition(phi,theta):
    Pos_SatellMatrix = np.zeros((4,3))
    Transmission_t = np.zeros(4)
    c = 299792.458
    rho=26570;
 
    for i in range(4):
        
        Pos_SatellMatrix[i][0] = rho*math.cos(phi[i])*math.cos(theta[i]) 
        Pos_SatellMatrix[i][1] = rho*math.cos(phi[i])*math.sin(theta[i])
        Pos_SatellMatrix[i][2] = rho*math.sin(phi[i])
        
        R = math.sqrt(Pos_SatellMatrix[i][0]**2 + Pos_SatellMatrix[i][1]**2 + (Pos_SatellMatrix[i][2]-6370)**2)
        
        Transmission_t[i] = 0.0001 + R/c
        
    return Pos_SatellMatrix, Transmission_t

In [5]:
# changes in the transmission time (+/- 10^-8)
def ChangeTranT(t0, N=15):    
    NewTranT = np.zeros((15,4))
    dt=10**(-8); #given value in question
    NewTranT[0]=t0;
    NewTranT[1]=[t0[0]-dt, t0[1]+dt, t0[2]+dt, t0[3]+dt];
    NewTranT[2]=[t0[0]+dt, t0[1]-dt, t0[2]+dt, t0[3]+dt];
    NewTranT[3]=[t0[0]+dt, t0[1]+dt, t0[2]-dt, t0[3]+dt];
    NewTranT[4]=[t0[0]+dt, t0[1]+dt, t0[2]+dt, t0[3]-dt];
    NewTranT[5]=[t0[0]-dt, t0[1]-dt, t0[2]+dt, t0[3]+dt];
    NewTranT[6]=[t0[0]+dt, t0[1]+dt, t0[2]-dt, t0[3]-dt];
    NewTranT[7]=[t0[0]-dt, t0[1]+dt, t0[2]+dt, t0[3]-dt];
    NewTranT[8]=[t0[0]+dt, t0[1]-dt, t0[2]-dt, t0[3]+dt];
    NewTranT[9]=[t0[0]-dt, t0[1]+dt, t0[2]-dt, t0[3]+dt];
    NewTranT[10]=[t0[0]+dt, t0[1]-dt, t0[2]+dt, t0[3]-dt];
    NewTranT[11]=[t0[0]+dt, t0[1]-dt, t0[2]-dt, t0[3]-dt];
    NewTranT[12]=[t0[0]-dt, t0[1]+dt, t0[2]-dt, t0[3]-dt];
    NewTranT[13]=[t0[0]-dt, t0[1]-dt, t0[2]+dt, t0[3]-dt];
    NewTranT[14]=[t0[0]-dt, t0[1]-dt, t0[2]-dt, t0[3]+dt];
    return NewTranT

In [6]:
RecieverPos #this is from Q1

array([-4.17727096e+01, -1.67891941e+01,  6.37005956e+03, -3.20156583e-03])

In [7]:
#Calculate Error Magnification Factor
def Cal_emf(RecieverPos,RecieverPosMatrix):
    c  = 299792.458
    dt = np.array([10**(-8), 10**(-8), 10**(-8), 10**(-8)]) #changes in time transmission
    du = RecieverPos - RecieverPosMatrix[0]
    normdu = LA.norm(du, np.inf)                #the forward error (or output error)
    normdt = c*LA.norm(dt, np.inf)              #the backward error (or input error)    
    return (normdu/normdt)

In [8]:
#Define array of Phi and Theta
phi   = [math.pi/8, 2*math.pi/8, 3*math.pi/8, 4*math.pi/8]
theta = [math.pi/2, math.pi, 3*math.pi/2, 2*math.pi] 

#Define Solution lists
RecieverPosMatrix = []
emf = []

#call functions to get satellites position and different tansmission time
Pos_SatelsMatrix, Transmission_t = SatellitePosition(phi,theta)
NewTranT =  ChangeTranT(Transmission_t)

# get Reciever Position in different tansmission time and return emf
for t_vector in NewTranT:

    InitialVector = np.array([0, 0, 6370, .0001])
    RecieverPos = RecieverPosition(Pos_SatelsMatrix, t_vector, InitialVector, N=20)
    RecieverPosMatrix.append(RecieverPos)
    emf.append(Cal_emf(RecieverPos,RecieverPosMatrix))

print('Matrix of Transmition Time Changes\n\n', NewTranT)
print('')
print('Position Matrix\n', *RecieverPosMatrix, sep="\n")
print('')
print('Error Magnification Factor List is:\n', *emf, sep="\n")
print('')
print('The Max Error Magnification Factor or Condition Number is:',np.max(emf))    

Matrix of Transmition Time Changes

 [[0.08295579 0.07522119 0.06957484 0.06747995]
 [0.08295578 0.0752212  0.06957485 0.06747996]
 [0.0829558  0.07522118 0.06957485 0.06747996]
 [0.0829558  0.0752212  0.06957483 0.06747996]
 [0.0829558  0.0752212  0.06957485 0.06747994]
 [0.08295578 0.07522118 0.06957485 0.06747996]
 [0.0829558  0.0752212  0.06957483 0.06747994]
 [0.08295578 0.0752212  0.06957485 0.06747994]
 [0.0829558  0.07522118 0.06957483 0.06747996]
 [0.08295578 0.0752212  0.06957483 0.06747996]
 [0.0829558  0.07522118 0.06957485 0.06747994]
 [0.0829558  0.07522118 0.06957483 0.06747994]
 [0.08295578 0.0752212  0.06957483 0.06747994]
 [0.08295578 0.07522118 0.06957485 0.06747994]
 [0.08295578 0.07522118 0.06957483 0.06747996]]

Position Matrix

[1.93263768e-12 3.10171641e-12 6.37000000e+03 1.00000000e-04]
[2.91857714e-03 1.41483329e-03 6.36999457e+03 9.99918958e-05]
[-7.18715868e-03  4.12041778e-12  6.37000000e+03  1.00010000e-04]
[ 5.90815207e-03 -9.41788180e-03  6.36998901e+03 

# Question 3

In [9]:
# different phi and theta range
phi   = [math.pi/8+0.01*math.pi/2, math.pi/8+0.02*math.pi/2, math.pi/8+0.03*math.pi/2, math.pi/8+0.04*math.pi/2];
theta = [math.pi/2+0.01*2*math.pi, math.pi/2+0.02*2*math.pi, math.pi/2+0.03*2*math.pi, math.pi/2+0.04*2*math.pi];

# define solution lists
RecieverPosMatrix = []
emf = []

#call functions to get satellites position and different tansmission time
Pos_SatelsMatrix, Transmission_t = SatellitePosition(phi,theta)
NewTranT =  ChangeTranT(Transmission_t)

# get Reciever Position in different tansmission time and return emf
for t_vector in NewTranT:
    InitialVector = np.array([0, 0, 6370, .0001])
    RecieverPos = RecieverPosition(Pos_SatelsMatrix,t_vector,InitialVector,N =20)
    RecieverPosMatrix.append(RecieverPos)
    emf.append(Cal_emf(RecieverPos,RecieverPosMatrix))

print('Matrix of Transmition Time Changes\n\n', NewTranT)
print('')
print('Position Matrix\n', *RecieverPosMatrix, sep="\n")
print('')
print('Error Magnification Factor List is:\n',*emf,sep="\n")
print('')
print('The Max Error Magnification Factor or Condition Number is:',np.max(emf)) 

Matrix of Transmition Time Changes

 [[0.08262638 0.0822979  0.08197043 0.08164408]
 [0.08262637 0.08229791 0.08197044 0.08164409]
 [0.08262639 0.08229789 0.08197044 0.08164409]
 [0.08262639 0.08229791 0.08197042 0.08164409]
 [0.08262639 0.08229791 0.08197044 0.08164407]
 [0.08262637 0.08229789 0.08197044 0.08164409]
 [0.08262639 0.08229791 0.08197042 0.08164407]
 [0.08262637 0.08229791 0.08197044 0.08164407]
 [0.08262639 0.08229789 0.08197042 0.08164409]
 [0.08262637 0.08229791 0.08197042 0.08164409]
 [0.08262639 0.08229789 0.08197044 0.08164407]
 [0.08262639 0.08229789 0.08197042 0.08164407]
 [0.08262637 0.08229791 0.08197042 0.08164407]
 [0.08262637 0.08229789 0.08197044 0.08164407]
 [0.08262637 0.08229789 0.08197042 0.08164409]]

Position Matrix

[8.44895734e-09 2.00790875e-09 6.37000000e+03 1.00000000e-04]
[-2.03962310e+01 -6.32482317e+00  6.28893939e+03  3.72909006e-05]
[6.14019752e+01 1.75317500e+01 6.61287011e+03 2.77744273e-04]
[-6.20662120e+01 -1.62324256e+01  6.12628578e+03 

# IV.) Conclusions

# V.) References